In [3]:
#%pylab inline
%matplotlib widget
%pylab
import numpy as np
import time
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, PathPatch
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from matplotlib import cm
import scipy.stats as ss
import scipy.signal as sig
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle as Rectangle
import mpl_toolkits.mplot3d.art3d as art3d
import random


Using matplotlib backend: module://ipympl.backend_nbagg
Populating the interactive namespace from numpy and matplotlib


/opt/anaconda3/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
import sys  
sys.path.insert(0, '/Users/JanviMadhani/satellite_planes')

import fortran_reader as fr

In [4]:
import pickle 
with open('/Users/JanviMadhani/satellite_planes/systems/MWsystems.pickle', 'rb') as handle:
    systems = pickle.loads(handle.read())

In [6]:
#print(systems)

## Define Plane Finder Functions

In [7]:
def dist(x,y,z,normal_vect,d):
    """
    distance between a point and a plane defined by a normal vector 
    
    """
    
    
    nx, ny, nz = normal_vect[0],normal_vect[1],normal_vect[2]
    num = np.abs(nx*x + ny*y + nz*z + d)
    den = np.sqrt(nx**2 + ny**2 + nz**2)
    
    dist = num/den
    
    return dist

In [8]:
def best_plane(system,level=1,n=10,mock=False,rand=False,verbose=False):
    """
    specifiy level = 1 for all true satellites
             level = 2 for galactic plane
             level = 0 *dummy 
    """
    if verbose:
        if not mock:
            print(f'Fitting plane to level {level} satellites... ')
 
    niter = n

    plane_finder = {}
    plane_finder['u1'] = []
    plane_finder['u2'] = []
    plane_finder['u3'] = []
    plane_finder['delta_s'] =[]
    plane_finder['rms_dist'] = []
    plane_finder['nx'] = []
    plane_finder['ny'] = []
    plane_finder['nz'] = []
    plane_finder['cos_theta'] = []
    plane_finder['phi'] = []

    for i in range(niter):


        u1 = random.uniform(0,1) #[0,1]  
        u2 = random.uniform(0,1) #[0,1]
        
        u3 = random.uniform(0,1) #sign

        plane_finder['u1'].append(u1)
        plane_finder['u2'].append(u2)
        plane_finder['u3'].append(u3)
            



        cos_theta = 2*u1 - 1   #makes sure cos_theta is bw -1,1

        sin_theta = np.sqrt(1-cos_theta**2)
        #randomly select sign of arccos 
        
        if u3 <= 0.5:
            sin_theta = -1*sin_theta

        
        phi = 2*np.pi*u2  #[0,2*pi] 
        
        plane_finder['cos_theta'].append(cos_theta)
        plane_finder['phi'].append(phi)




        nx = np.cos(phi)*sin_theta
        ny = np.sin(phi)*sin_theta
        nz = cos_theta
        n = np.array([nx,ny,nz])
        mag_n = np.linalg.norm(n)
        unit_n = n/mag_n
        #print(unit_n)
        
        plane_finder['nx'].append(unit_n[0])
        plane_finder['ny'].append(unit_n[1])
        plane_finder['nz'].append(unit_n[2])

        if mock:
            x0 = system['MW_px']
            y0 = system['MW_py']
            z0 = system['MW_pz']
        elif rand:
            x0 = system['MW_px']
            y0 = system['MW_py']
            z0 = system['MW_pz']
            
        else:
            x0 = systems[system]['MW_px'][0]
            y0 = systems[system]['MW_py'][0]
            z0 = systems[system]['MW_pz'][0]

        gal_center = np.array([x0,y0,z0])

        d = np.dot(-gal_center,unit_n)
        #print('A,B,C,D:',unit_n[0],unit_n[1],unit_n[2],d)

        #equation of plane (Ax + By + Cz + D = 0): unit_n[0] * x + unit_n[1] * y  + unit_n[2]*z + d = 0 

        #calculate distances

        distances = []
        #nsat = len(system['sat_pxs'])
        if mock:
            nsats = len(system['sat_pxs'])
            for k in range(len(system['sat_pxs'])):
                x,y,z = system['sat_pxs'][k],system['sat_pys'][k],system['sat_pzs'][k]
                s = dist(x,y,z,unit_n,d)
                distances.append(s)
                
        elif rand:
            nsats = len(system['sat_px'])
            for k in range(len(system['sat_px'])):
                x,y,z = system['sat_px'][k],system['sat_py'][k],system['sat_pz'][k]
                s = dist(x,y,z,unit_n,d)
                distances.append(s)
            
                
        else:
            

            if level == 1:
                level_one_sats = np.where(systems[system]['sat_levels'] == 1)
                nsats = len(level_one_sats[0]) 
                for k in range(nsats):
                    x,y,z = systems[system]['sat_pxs'][level_one_sats][k],systems[system]['sat_pys'][level_one_sats][k],systems[system]['sat_pzs'][level_one_sats][k]
                    s = dist(x,y,z,unit_n,d)
                    distances.append(s)
            elif level == 2:
                level_two_sats = np.where(systems[system]['sat_levels'] == 2)
                nsats = len(level_two_sats[0])
                for k in range(nsats):
                    x,y,z = systems[system]['sat_pxs'][level_two_sats][k],systems[system]['sat_pys'][level_two_sats][k],systems[system]['sat_pzs'][level_two_sats][k]
                    s = dist(x,y,z,unit_n,d)
                    distances.append(s)
           
            elif level == 3:
                level_three_sats = np.where(systems[system]['sat_levels'] == 3)
                nsats = len(level_three_sats[0])
                for k in range(nsats):
                    x,y,z = systems[system]['sat_pxs'][level_three_sats][k],systems[system]['sat_pys'][level_three_sats][k],systems[system]['sat_pzs'][level_three_sats][k]
                    s = dist(x,y,z,unit_n,d)
                    distances.append(s)
            else:
                for k in range(len(systems[system]['sat_pxs'])):
                    x,y,z = systems[system]['sat_pxs'][k],systems[system]['sat_pys'][k],systems[system]['sat_pzs'][k]
                    s = dist(x,y,z,unit_n,d)
                    distances.append(s)
                    
                
              
        distances = np.asarray(distances)

        rms = np.sqrt(np.mean(distances**2))
        #print(rms)
        plane_finder['delta_s'].append(distances)
        plane_finder['rms_dist'].append(rms)
        
        
    #find minimum rms and corresponding u1, u2 
        
    u1_a = np.asarray(plane_finder['u1'])
    #print(len(u1_a))
    u2_a = np.asarray(plane_finder['u2'])
    u3_a = np.asarray(plane_finder['u3'])
    rms_a = plane_finder['rms_dist']

    #print(rms_a)
    
    cos_theta_a = np.asarray(plane_finder['cos_theta'])
    phi_a = np.asarray(plane_finder['phi'])
    #print(rms_a)

    best_plane = np.argmin(rms_a)
        
    
    best_rms = plane_finder['rms_dist'][best_plane]
        
    if verbose:
        print(f'Fitting to {nsats} satellites...')  
        print('best plane index:',best_plane)
        print('Best plane has:')
        print('Cos(theta):', cos_theta_a[best_plane])
        print('Phi',phi_a[best_plane])
        print(f'u1 = {u1_a[best_plane]}; u2 = {u2_a[best_plane]}')
        print(f'Best rms = {best_rms}')
    
    return u1_a[best_plane],u2_a[best_plane],u3_a[best_plane],plane_finder['nx'],plane_finder['ny'],plane_finder['nz'],plane_finder['rms_dist'],best_rms,plane_finder['delta_s'],cos_theta_a[best_plane]
        
        
        



In [66]:
### Write evolutionary algorithm

def evolutionary_plane_finder(n_iter,n_start,n_erase,n_mutants,level=1):
    
    plane_finder = {}
    plane_finder['rms_dist'] = []
    plane_finder['nx'] = []
    plane_finder['ny'] = []
    plane_finder['nz'] = []
    plane_finder['u1'] = []
    plane_finder['u2'] = []
    plane_finder['u3'] = []
    

    #start with initial population 
    for k in range(n_start):

        u1 = random.uniform(0,1) #[0,1]  
        u2 = random.uniform(0,1) #[0,1]

        u3 = random.uniform(0,1) #sign
        plane_finder['u1'].append(u1)
        plane_finder['u2'].append(u2)
        plane_finder['u3'].append(u3)

        cos_theta = 2*u1 - 1   #makes sure cos_theta is bw -1,1

        sin_theta = np.sqrt(1-cos_theta**2)
        #randomly select sign of arccos 

        if u3 <= 0.5:
            sin_theta = -1*sin_theta


        phi = 2*np.pi*u2  #[0,2*pi] 

        nx = np.cos(phi)*sin_theta
        ny = np.sin(phi)*sin_theta
        nz = cos_theta
        n = np.array([nx,ny,nz])
        mag_n = np.linalg.norm(n)
        unit_n = n/mag_n
        
        plane_finder['nx'].append(unit_n[0])
        plane_finder['ny'].append(unit_n[1])
        plane_finder['nz'].append(unit_n[2])

        x0 = systems[system]['MW_px'][0]
        y0 = systems[system]['MW_py'][0]
        z0 = systems[system]['MW_pz'][0]

        gal_center = np.array([x0,y0,z0])

        d = np.dot(-gal_center,unit_n)
        
        distances = []

        level_sats = np.where(systems[system]['sat_levels'] == level)
        nsats = len(level_sats[0]) 
        for j in range(nsats):
            x,y,z = systems[system]['sat_pxs'][level_one_sats][k],systems[system]['sat_pys'][level_one_sats][k],systems[system]['sat_pzs'][level_one_sats][k]
            s = dist(x,y,z,unit_n,d)
            distances.append(s)
            
                distances = np.asarray(distances)

        rms = np.sqrt(np.mean(distances**2))
        #plane_finder['delta_s'].append(distances)
        plane_finder['rms_dist'].append(rms)
        
    #now, evolve n_iter times
    i = 0 
    while i <= n_iter:
        
        #eliminate n_erase
        #first sort list of rms to be from min to max so you can eliminate n_erase
        
        sorted_rms = argsort(plane_finder['rms_dist'])
        for erase_index in range(n_erase):
            i_to_erase = sorted_rms[-1]
            


In [81]:
rms = [1,2,4,0,6,4,2,6,8,30,1,3,5,2,1,25,12,46]
u1 = [1,2,4,0,6,4,2,6,8,30,1,3,5,2,1,25,12,46]
u2 = [1,2,4,0,6,4,2,6,8,30,1,3,5,2,1,25,12,46]
u3 = [1,2,4,0,6,4,2,6,8,30,1,3,5,2,1,25,12,46]

nx = [1,2,4,0,6,4,2,6,8,30,1,3,5,2,1,25,12,46]
ny = [1,2,4,0,6,4,2,6,8,30,1,3,5,2,1,25,12,46]
nz = [1,2,4,0,6,4,2,6,8,30,1,3,5,2,1,25,12,46]

sorted_rms = argsort(rms)
print(rms)
print(sorted_rms)

n_erase = 3
n_iter = 10
n_avg_mutants = 1
n_mutants = n_erase - n_avg_mutants
i = 0

while i <=n_iter:
    print(i)
    to_erase = []
    for e in range(n_erase):
        i_to_erase = sorted_rms[-(e+1)]
        to_erase.append(i_to_erase)
        
    for m in range(n_avg_mutants):
        
        #add the average of all the erased
        
        avg_u1 = np.mean([u1[i] for i in to_erase])
        avg_u2 = np.mean([u2[i] for i in to_erase])
        avg_u3 = np.mean([u3[i] for i in to_erase])
        
        cos_theta_avg = 2*avg_u1 - 1   #makes sure cos_theta is bw -1,1

        sin_theta_avg = np.sqrt(1-cos_theta_avg**2)
        #randomly select sign of arccos 

        if avg_u3 <= 0.5:
            sin_theta_avg = -1*sin_theta_avg


        phi_avg = 2*np.pi*u2_avg  #[0,2*pi] 

        nx_avg = np.cos(phi_avg)*sin_theta_avg
        ny_avg = np.sin(phi_avg)*sin_theta_avg
        nz_avg = cos_theta_avg
        n_avg = np.array([nx_avg,ny_avg,nz_avg])
        mag_n_avg = np.linalg.norm(n_avg)
        unit_n_avg = n_avg/mag_n_avg
        
        #now that you're done using to be erased data, erase it
        u1.pop(i_to_erase)
        u2.pop(i_to_erase)
        u3.pop(i_to_erase)
        
        
        #add them to the population
        
        #add true mutants to the population
        
        #find the distance/rms of these new guys and add them to the rms list
        
        
    
    

        
        
        
    i+=1


[1, 2, 4, 0, 6, 4, 2, 6, 8, 30, 1, 3, 5, 2, 1, 25, 12, 46]
[ 3  0 14 10  1  6 13 11  2  5 12  4  7  8 16 15  9 17]
0
[1, 2, 4, 0, 6, 4, 2, 6, 8, 30, 1, 3, 5, 2, 1, 25, 12]
[1, 2, 4, 0, 6, 4, 2, 6, 8, 30, 1, 3, 5, 2, 1, 25, 12]
[1, 2, 4, 0, 6, 4, 2, 6, 8, 30, 1, 3, 5, 2, 1, 25, 12]


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in sqrt


NameError: name 'u2_' is not defined

In [9]:
"""level_one_sats = np.where(systems[syst]['sat_levels'] == 2)
print(systems[syst]['sat_levels'])
print(len(level_one_sats[0]))
"""
#for k in range(len(level_one_sats[0])):
    #print(k)

"level_one_sats = np.where(systems[syst]['sat_levels'] == 2)\nprint(systems[syst]['sat_levels'])\nprint(len(level_one_sats[0]))\n"

In [10]:
def get_plane(u1,u2,u3,system,mock=False):



    cos_theta = 2*u1 - 1   #makes sure cos_phi is bw 0,1

    sin_theta = np.sqrt(1-cos_theta**2)
    #randomly select sign of arccos 

    if u3 <= 0.5:
        sin_theta = -1*sin_theta


    phi = 2*np.pi*u2  #[-pi,pi]  

    nx = np.cos(phi)*sin_theta
    ny = np.sin(phi)*sin_theta
    nz = cos_theta
    n = np.array([nx,ny,nz])
    mag_n = np.linalg.norm(n)
    unit_n = n/mag_n
    
    if mock:
        x0 = system['MW_px']
        y0 = system['MW_py']
        z0 = system['MW_pz']
    else: #both random dist and actual dist are centered at same central galaxy
        x0 = systems[system]['MW_px'][0]
        y0 = systems[system]['MW_py'][0]
        z0 = systems[system]['MW_pz'][0]

    gal_center = np.array([x0,y0,z0])

    d = np.dot(-gal_center,unit_n)

    if mock:
        xx,yy = np.meshgrid(np.linspace(-1,1,100),np.linspace(-1,1,100))
        
    else:
        #"""
        xx, yy = np.meshgrid(np.linspace(systems[system]['halo_px']-systems[system]['halo_rvir']*8e-1,systems[system]['halo_px']+systems[system]['halo_rvir']*8e-1,100),
        np.linspace(systems[system]['halo_py']-systems[system]['halo_rvir']*8e-1,systems[system]['halo_py']+systems[system]['halo_rvir']*8e-1,100))
        #"""
        
        """        
        min_x,max_x= np.min(systems[system]['sat_pxs']),np.max(systems[system]['sat_pxs'])
        min_y,max_y= np.min(systems[system]['sat_pys']),np.max(systems[system]['sat_pys'])
        xx, yy = np.meshgrid(np.linspace(min_x,max_x,100),np.linspace(min_y,max_y,100))
        """

    # calculate corresponding z
    z = (-unit_n[0] * xx - unit_n[1] * yy - d) * 1. /unit_n[2]
    #z = (- unit_n[1]*xx)/unit_n[2] - (unit_n[0]*yy)/unit_n[2]
    
    return z,xx,yy,unit_n








## Make a mock system

In [11]:
nfake = 15

x_mock = [random.uniform(-1,1) for i in range(nfake)]
y_mock = [random.uniform(-1,1) for i in range(nfake)] #restrict y-range to make it a plane
z_mock = [random.uniform(-0.2,0.2) for i in range(nfake)]
MW_x,MW_y,MW_z = 0,0,0 #place the central in the middle
MW_rvir = 0.3
sat_rvir = [random.uniform(0,0.2) for i in range(10)]

mock_system = {}
mock_system['sat_pxs']=x_mock
mock_system['sat_pys']=y_mock
mock_system['sat_pzs']=z_mock
mock_system['sat_rvirs'] = sat_rvir
mock_system['MW_px']=MW_x
mock_system['MW_py']=MW_y
mock_system['MW_pz']=MW_z
mock_system['MW_rvir']=MW_rvir

x_mock = np.asarray(x_mock)
y_mock = np.asarray(y_mock)
z_mock = np.asarray(z_mock)
sat_rvir = np.asarray(sat_rvir)



In [12]:
fig = plt.figure(figsize=[8,8])
ax = plt.axes(projection='3d')



imsats = ax.scatter3D(mock_system['sat_pxs'],mock_system['sat_pys'],mock_system['sat_pzs'],
                      s=np.asarray(mock_system['sat_rvirs'])*1e3,c='rosybrown',edgecolors='indianred',alpha=0.4,label='Satellites')
imcentral = ax.scatter3D(mock_system['MW_px'],mock_system['MW_py'],mock_system['MW_pz'],
                      s=np.asarray(mock_system['MW_rvir'])*1e3,c='slateblue',edgecolors='darkblue',alpha=0.4,label='Central')


central = Circle((mock_system['MW_px'],mock_system['MW_py']), radius=mock_system['MW_rvir'],color='gold',alpha=0.4,label='Rvir of MW')
ax.add_patch(central)
art3d.pathpatch_2d_to_3d(central, mock_system['MW_pz'], zdir="z")

#plot the plane
#plane = ax.plot_surface(xx,yy, z_best,color='k' ,alpha=0.4)


ax.set_title('MW type Halo System, scaled by Virial Radius of Objects')
ax.set_xlim(-1,1)
ax.set_ylim(-1,1)
ax.set_zlim(-1,1)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
plt.legend(loc="upper right", markerscale=.05)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Run Plane Finder on Mock System

In [13]:
best_u1,best_u2,best_u3,nx,ny,nz,rms,best_rms,delta_s,best_cos_theta = best_plane(system=mock_system,n=2000,mock=True,verbose=True)

Fitting to 15 satellites...
best plane index: 157
Best plane has:
Cos(theta): -0.9971896024063938
Phi 0.8932700731532354
u1 = 0.0014051987968031199; u2 = 0.14216834765839637
Best rms = 0.14030461707120503


In [14]:
#print(delta_s)

In [15]:
z_best,xx,yy,unit_n = get_plane(u1=best_u1,u2=best_u2,u3=best_u3,system=mock_system,mock=True)

In [16]:
fig = plt.figure(figsize=[8,8])
ax = plt.axes(projection='3d')



imsats = ax.scatter3D(mock_system['sat_pxs'],mock_system['sat_pys'],mock_system['sat_pzs'],
                      s=np.asarray(mock_system['sat_rvirs'])*1e3,c='rosybrown',edgecolors='indianred',alpha=0.4,label='Satellites')
imcentral = ax.scatter3D(mock_system['MW_px'],mock_system['MW_py'],mock_system['MW_pz'],
                      s=np.asarray(mock_system['MW_rvir'])*1e3,c='slateblue',edgecolors='darkblue',alpha=0.4,label='Central')

normal = ax.quiver(mock_system['MW_px'],mock_system['MW_py'],mock_system['MW_pz'],
                       unit_n[0],unit_n[1],unit_n[2],color='red', length=0.4, normalize=True,label='Normal')

"""for i in range(nfake):
    ax.text(mock_system['sat_pxs'][i],mock_system['sat_pys'][i],mock_system['sat_pzs'][i],
           "%.2f" %delta_s[0][i])
"""
#plane_points = ax.scatter3D(xx,yy,z_best,s=2)

central = Circle((mock_system['MW_px'],mock_system['MW_py']), radius=mock_system['MW_rvir'],color='gold',alpha=0.4,label='Rvir of MW')
ax.add_patch(central)
art3d.pathpatch_2d_to_3d(central, mock_system['MW_pz'], zdir="z")

#plot the plane
plane = ax.plot_surface(xx,yy, z_best,color='plum' ,alpha=0.4)


ax.set_title('MW type Halo System, scaled by Virial Radius of Objects')
ax.set_xlim(-1,1)
ax.set_ylim(-1,1)
ax.set_zlim(-1,1)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
plt.legend(loc="upper right", markerscale=.05)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
#print('distance',delta_s)

#print('location of satellites',mock_system['sat_pxs'],mock_system['sat_pys'],mock_system['sat_pzs'])


In [18]:
#print(rms)

In [19]:
# Creating histogram
#angles = np.asarray(angles)
n= len(rms)
fig, ax = plt.subplots(1, 1,
                        figsize =(8,5), 
                        tight_layout = True)

histbins = 25
  
ax.hist(rms, density=False,bins =histbins,ec='purple',fc='thistle')
ax.axvline(x=0.023262901924825725)
ax.set_title(f'RMS distance')
ax.set_xlabel(r'RMS')
#ax.set_xlim(0,1)
  
# Show plot
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
fig = plt.figure(figsize=[8,8])
ax = plt.axes(projection='3d')

sphere_points = ax.scatter3D(nx,ny,nz)

ax.set_xlim(-1,1)
ax.set_ylim(-1,1)
ax.set_zlim(-1,1)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Choose a System    

In [23]:
#print(systems)
for syst in systems:
    print(syst['halo_ID'])

1300
10888
1420
562
1387
27406
20303
3330
24660
20579
22148
5484
23497
3365
674
5482
3509
5485
27785
19931
1673
4241
13426
4009
9410
5497
19740
5559
1344
25400
4088
11068
474
5485
8289
593
8162
10885
27182
19675
24368
8281
1849
1774
1363
8291
466
27489
13768
15021
13675
2350
13241
5181
1673


In [24]:
syst = 46 #29 #4 could have planes #17!! (companions = 29 and 47) (17 has good plane due to merger)
#43 has a plane with few satellites, 39 has good plane, 46 has a good plane and system
#46 to resimulate

In [25]:
print('Halo ID:', systems[syst]['halo_ID'])
print('Rvir:',systems[syst]['halo_rvir']) #mpc
print('Position',systems[syst]['halo_px'],systems[syst]['halo_py'],systems[syst]['halo_pz'])


Halo ID: 466
Rvir: [0.33019677]
Position [-2.77927661] [-0.26692456] [-9.50934982]


In [26]:
## quick look at system


fig = plt.figure(figsize=[8,6])
ax = plt.axes(projection='3d')

M_to_k = 1000
MW_x,MW_y,MW_z = systems[syst]['MW_px'],systems[syst]['MW_py'],systems[syst]['MW_pz']
nsats = len(systems[syst]['sat_pxs'])

scaleby = 4


for i in range(nsats):
    sat = Circle(((systems[syst]['sat_pxs'][i]-MW_x)*M_to_k, (systems[syst]['sat_pys'][i] - MW_y)*M_to_k), radius=systems[syst]['sat_rvirs'][i]*M_to_k,color='indianred',alpha=0.4)
    ax.add_patch(sat)
    art3d.pathpatch_2d_to_3d(sat, (systems[syst]['sat_pzs'][i]-MW_z)*M_to_k, zdir="z")

imsats = ax.scatter3D((systems[syst]['sat_pxs'] - MW_x)*M_to_k,(systems[syst]['sat_pys']- MW_y)*M_to_k,(systems[syst]['sat_pzs'] - MW_z)*M_to_k,
                      s=systems[syst]['sat_rvirs']*M_to_k*scaleby**2,c=np.log10(systems[syst]['sat_mvirs']),edgecolors='indianred',alpha=0.4,label='Satellites')
imcentral = ax.scatter3D((systems[syst]['MW_px']- MW_x)*M_to_k,(systems[syst]['MW_py']- MW_y)*M_to_k ,(systems[syst]['MW_pz'] - MW_z)*M_to_k,
                      s=systems[syst]['MW_rvir']*M_to_k*scaleby**2,c='slateblue',edgecolors='darkblue',alpha=0.4,label='Central')

central = Circle(((systems[syst]['MW_px'] - MW_x)*M_to_k, (systems[syst]['MW_py'] - MW_y)*M_to_k), radius=systems[syst]['MW_rvir']*M_to_k,color='slateblue',alpha=0.4,label='Rvir of MW')
ax.add_patch(central)
art3d.pathpatch_2d_to_3d(central, (systems[syst]['MW_pz']-MW_z)*M_to_k, zdir="z")




#ax.set_title(r'MW type Satellite System, $N_{sats}$ = 7',y=1.15)
plt.colorbar(imsats,label=r'Mass of Satellites [$log_{10} M_\odot$]')
ax.autoscale('False')
ax.set_xlabel('X [kpc]')
ax.set_ylabel('Y [kpc]')
ax.set_zlabel('Z [kpc]')
plt.legend(loc="upper left", markerscale=.4)

extent = 500 #kpc
ax.set_xlim(-extent,extent)
ax.set_ylim(-extent,extent)
ax.set_zlim(-extent,extent)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Look at Histogram

In [27]:
systems[syst]['sat_pxs']

array([-2.2887782 , -2.73628868, -2.79496687, -2.79161147, -2.86498459,
       -2.66406764, -2.85265409, -2.64141595, -2.80318028, -2.72596213,
       -2.85747541, -2.76134609, -2.79438061, -2.76566846, -2.74600907,
       -2.7698527 , -2.79555254, -2.75688145, -2.79844343, -2.81002326,
       -2.82597265, -2.78469711, -2.80675467, -2.80048193, -2.80698607,
       -2.8123747 , -2.80185577, -2.79790403, -2.75895631, -2.79589921,
       -2.77990032, -2.81195232, -2.80334204, -2.79493081, -2.77991058,
       -2.77650108, -2.76961101, -2.80665535, -2.78055832, -2.7853655 ,
       -2.79434323, -2.80388037, -2.80304486, -2.79118314, -2.80007904,
       -2.79232275, -2.79701016, -2.8050056 , -2.8004197 , -2.77623516,
       -2.79752477, -2.80077371, -2.7742547 , -2.79660801, -2.80632219,
       -2.77720812, -2.77912098, -2.75716461, -2.79762882, -2.79647421,
       -2.79532777, -2.77991808, -2.80018956, -2.77549657, -2.79356536,
       -2.79732403, -2.78300399, -2.79487053, -2.7997939 , -2.79

In [28]:
# Creating histogram
#angles = np.asarray(angles)
nsats = len(systems[syst]['sat_pxs'])
fig, ax = plt.subplots(1, 1,
                        figsize =(8,5), 
                        tight_layout = True)

histbins = 25
  
ax.hist(np.abs(systems[syst]['cos']), density=False,bins =histbins,ec='purple',fc='thistle')
ax.set_title(f'Angular Distribution of {nsats} Satellites')
ax.set_xlabel(r'abs(Cos $\theta$)')
ax.set_xlim(0,1)
  
# Show plot
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

 ## Run Plane Finder

In [29]:
best_u1,best_u2,best_u3,nx,ny,nz,rms,best_rms,delta_s,best_cos_theta = best_plane(system=syst,level=1,n=5000,verbose=True)

Fitting plane to level 1 satellites... 
Fitting to 14 satellites...
best plane index: 3957
Best plane has:
Cos(theta): 0.6473488765287263
Phi 4.025850577231115
u1 = 0.8236744382643632; u2 = 0.6407340195156921
Best rms = 0.04286656347317104


In [30]:
z_best,xx,yy,unit_n = get_plane(u1=best_u1,u2=best_u2,u3=best_u3,system=syst)

In [31]:
fig = plt.figure(figsize=[8,6])
ax = plt.axes(projection='3d')

nsats = len(systems[syst]['sat_pxs'])

imhalo = ax.scatter3D(systems[syst]['halo_px'],systems[syst]['halo_py'],systems[syst]['halo_pz'],
                      s=systems[syst]['halo_rvir']*100**2,c='cornflowerblue',edgecolors='royalblue',alpha=0.4,label='MW type DM Halo')

for i in range(nsats):
    sat = Circle((systems[syst]['sat_pxs'][i], systems[syst]['sat_pys'][i]), radius=systems[syst]['sat_rvirs'][i],color='black',alpha=0.4)
    ax.add_patch(sat)
    art3d.pathpatch_2d_to_3d(sat, systems[syst]['sat_pzs'][i], zdir="z")

#color by mass
"""
imsats = ax.scatter3D(systems[syst]['sat_pxs'],systems[syst]['sat_pys'],systems[syst]['sat_pzs'],
                      s=systems[syst]['sat_rvirs']*100**2,c=np.log10(systems[syst]['sat_mvirs']),edgecolors='indianred',alpha=0.4,label='Satellites')
"""

#color by line of sight = x, velocity
imsats = ax.scatter3D(systems[syst]['sat_pxs'],systems[syst]['sat_pys'],systems[syst]['sat_pzs'],
                      s=systems[syst]['sat_rvirs']*100**2,c=systems[syst]['sat_vxs']-systems[syst]['MW_vx'],cmap='seismic',alpha=0.8,label='Satellites')

imcentral = ax.scatter3D(systems[syst]['MW_px'],systems[syst]['MW_py'],systems[syst]['MW_pz'],
                      s=systems[syst]['MW_rvir']*100**2,c='slateblue',edgecolors='darkblue',alpha=0.4,label='Central')
spin = ax.quiver(systems[syst]['MW_px'],systems[syst]['MW_py'],systems[syst]['MW_pz'],
                       systems[syst]['MW_lx'],systems[syst]['MW_ly'],systems[syst]['MW_lz'],color='black', length= 3e-2, normalize=True,label='Spin')
normal = ax.quiver(systems[syst]['MW_px'],systems[syst]['MW_py'],systems[syst]['MW_pz'],
                       unit_n[0],unit_n[1],unit_n[2],color='red', length=3e-2, normalize=True,label='Normal')
central = Circle((systems[syst]['MW_px'], systems[syst]['MW_py']), radius=systems[syst]['MW_rvir'],color='slateblue',alpha=0.4,label='Rvir of MW')
ax.add_patch(central)
art3d.pathpatch_2d_to_3d(central, systems[syst]['MW_pz'], zdir="z")

#plot the plane
plane = ax.plot_surface(xx,yy, z_best,color='k' ,alpha=0.4)


ax.set_title(r'MW type Satellite System, $N_{sats}$ = 7',y=1.15)
#plt.colorbar(imsats,label=r'Mass of Satellites [$log_{10} M_\odot$]')
plt.colorbar(imsats,label=r'Velocity of Satellites [km/s]')
ax.autoscale('False')
ax.set_xlabel('X [Mpc]')
ax.set_ylabel('Y [Mpc]')
ax.set_zlabel('Z [Mpc]')
plt.legend(loc="lower left", markerscale=.05)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [54]:
## Figure for presentation

fig = plt.figure(figsize=[8,6])
ax = plt.axes(projection='3d')

M_to_k = 1000
MW_x,MW_y,MW_z = systems[syst]['MW_px'],systems[syst]['MW_py'],systems[syst]['MW_pz']
nsats = len(systems[syst]['sat_pxs'])

scaleby = 3


for i in range(nsats):
    sat = Circle(((systems[syst]['sat_pxs'][i]-MW_x)*M_to_k, (systems[syst]['sat_pys'][i] - MW_y)*M_to_k), radius=systems[syst]['sat_rvirs'][i]*M_to_k,color='indianred',alpha=0.4)
    ax.add_patch(sat)
    art3d.pathpatch_2d_to_3d(sat, (systems[syst]['sat_pzs'][i]-MW_z)*M_to_k, zdir="z")
imsats = ax.scatter3D((systems[syst]['sat_pxs'] - MW_x)*M_to_k,(systems[syst]['sat_pys']- MW_y)*M_to_k,(systems[syst]['sat_pzs'] - MW_z)*M_to_k,
                      s=systems[syst]['sat_rvirs']*M_to_k*scaleby**2,c=np.log10(systems[syst]['sat_mvirs']),edgecolors='indianred',alpha=0.4,label='Satellites')
imcentral = ax.scatter3D((systems[syst]['MW_px']- MW_x)*M_to_k,(systems[syst]['MW_py']- MW_y)*M_to_k ,(systems[syst]['MW_pz'] - MW_z)*M_to_k,
                      s=systems[syst]['MW_rvir']*M_to_k*scaleby**2,c='slateblue',edgecolors='darkblue',alpha=0.4,label='Central')
#spin = ax.quiver((systems[syst]['MW_px']-MW_x)*M_to_k,(systems[syst]['MW_py']-MW_y)*M_to_k,(systems[syst]['MW_pz']-MW_z)*M_to_k,
                      # systems[syst]['MW_lx'],systems[syst]['MW_ly'],systems[syst]['MW_lz'],color='black', length= 30, normalize=True,label='Spin')

central = Circle(((systems[syst]['MW_px'] - MW_x)*M_to_k, (systems[syst]['MW_py'] - MW_y)*M_to_k), radius=systems[syst]['MW_rvir']*M_to_k,color='slateblue',alpha=0.4,label='Rvir of MW')
ax.add_patch(central)
art3d.pathpatch_2d_to_3d(central, (systems[syst]['MW_pz']-MW_z)*M_to_k, zdir="z")

#plot the plane
plane = ax.plot_surface((xx-MW_x)*M_to_k,(yy-MW_y)*M_to_k, (z_best-MW_z)*M_to_k,color='k' ,alpha=0.4)


ax.set_title(r'MW type Satellite System, $N_{nsats}$ =' + f'{nsats}',y=1.15)
plt.colorbar(imsats,label=r'Mass of Satellites [$log_{10} M_\odot$]')
ax.autoscale('False')
ax.set_xlabel('X [kpc]')
ax.set_ylabel('Y [kpc]')
ax.set_zlabel('Z [kpc]')
plt.legend(loc="upper left", markerscale=.4)

#"""
extent = 500 #kpc
ax.set_xlim(-extent,extent)
ax.set_ylim(-extent,extent)
ax.set_zlim(-extent,extent)
#"""

plt.show()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [63]:
## Figure for presentation

fig = plt.figure(figsize=[6,6])
ax = plt.axes(projection='3d')

M_to_k = 1000
MW_x,MW_y,MW_z = systems[syst]['MW_px'],systems[syst]['MW_py'],systems[syst]['MW_pz']
nsats = len(systems[syst]['sat_pxs'])
level1 = len(np.where(systems[syst]['sat_levels'] == 1)[0])


scaleby = 3


for i in range(nsats):
    sat = Circle(((systems[syst]['sat_pxs'][i]-MW_x)*M_to_k, (systems[syst]['sat_pys'][i] - MW_y)*M_to_k), radius=systems[syst]['sat_rvirs'][i]*M_to_k,color='black',alpha=0.4)
    ax.add_patch(sat)
    art3d.pathpatch_2d_to_3d(sat, (systems[syst]['sat_pzs'][i]-MW_z)*M_to_k, zdir="z")
    color = systems[syst]['sat_vxs'][i]-systems[syst]['MW_vx']
    if color > 0:
        ax.scatter3D((systems[syst]['sat_pxs'][i] - MW_x)*M_to_k,(systems[syst]['sat_pys'][i]- MW_y)*M_to_k,(systems[syst]['sat_pzs'][i] - MW_z)*M_to_k,
                      s=systems[syst]['sat_rvirs'][i]*M_to_k*scaleby**2,c='blue',alpha=0.8)
    else:
        ax.scatter3D((systems[syst]['sat_pxs'][i] - MW_x)*M_to_k,(systems[syst]['sat_pys'][i]- MW_y)*M_to_k,(systems[syst]['sat_pzs'][i] - MW_z)*M_to_k,
                      s=systems[syst]['sat_rvirs'][i]*M_to_k*scaleby**2,c='red',alpha=0.8)
    
    
    

imcentral = ax.scatter3D((systems[syst]['MW_px']- MW_x)*M_to_k,(systems[syst]['MW_py']- MW_y)*M_to_k ,(systems[syst]['MW_pz'] - MW_z)*M_to_k,
                      s=systems[syst]['MW_rvir']*M_to_k*scaleby**2,c='slateblue',edgecolors='darkblue',alpha=0.4,label='Central')


central = Circle(((systems[syst]['MW_px'] - MW_x)*M_to_k, (systems[syst]['MW_py'] - MW_y)*M_to_k), radius=systems[syst]['MW_rvir']*M_to_k,color='slateblue',alpha=0.4,label='Rvir of MW')
ax.add_patch(central)
art3d.pathpatch_2d_to_3d(central, (systems[syst]['MW_pz']-MW_z)*M_to_k, zdir="z")

#plot the plane
plane = ax.plot_surface((xx-MW_x)*M_to_k,(yy-MW_y)*M_to_k, (z_best-MW_z)*M_to_k,color='k' ,alpha=0.4)


#ax.set_title(r'MW type Satellite System, $N_{sats}$ =' + f'{level1}',y=1.15)

ax.autoscale('False')
ax.set_xlabel('X [kpc]')
ax.set_ylabel('Y [kpc]')
ax.set_zlabel('Z [kpc]')


#"""
extent = 500 #kpc
ax.set_xlim(-extent,extent)
ax.set_ylim(-extent,extent)
ax.set_zlim(-extent,extent)
#"""

plt.show()











/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [50]:
#for sys in systems:
    #print(np.log10(sys['MW_mvir']))

In [34]:
fig = plt.figure(figsize=[8,8])
ax = plt.axes(projection='3d')

sphere_points = ax.scatter3D(nx,ny,nz)

ax.set_xlim(-1,1)
ax.set_ylim(-1,1)
ax.set_zlim(-1,1)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
"""nsats = 30
separation = []

for i,j in zip(range(nsats),range(nsats)):
    r1 = systems[syst]['sat_pxs'][i]
    while j>i:
        r2 = systems[syst]['sat_pxs'][j]
        sep = r2 - r1
        separation.append(sep)
        """

"nsats = 30\nseparation = []\n\nfor i,j in zip(range(nsats),range(nsats)):\n    r1 = systems[syst]['sat_pxs'][i]\n    while j>i:\n        r2 = systems[syst]['sat_pxs'][j]\n        sep = r2 - r1\n        separation.append(sep)\n        "

## Check for Anisotropy

### Generate Random Distribution, Preserving Separation Vector

In [36]:
def rand_angle():
    u1 = random.uniform(0,1) #[0,1]  
    u2 = random.uniform(0,1) #[0,1]

    u3 = random.uniform(0,1) #sign

    #spherical_isotropy['u1'].append(u1)
    #spherical_isotropy['u2'].append(u2)
    #spherical_isotropy['u3'].append(u3)

    cos_theta = 2*u1 - 1   #makes sure cos_theta is bw -1,1


    sin_theta = np.sqrt(1-cos_theta**2)

    #randomly select sign of arccos 

    if u3 <= 0.5:
        sin_theta = -1*sin_theta


    phi = 2*np.pi*u2  #[0,2*pi] 
    
    return cos_theta, sin_theta, phi

    

def rand_spherical_dist(system,level=1):
 
    niter = n
    
    #best_u1,best_u2,best_u3,nx,ny,nz,rms,delta_s = best_plane(system,level=1,n=5000)

    spherical_isotropy = {}
    #spherical_isotropy['cos theta'] = []
    #spherical_isotropy['phi'] = []
    spherical_isotropy['sat_x'] = []
    spherical_isotropy['sat_y'] = []
    spherical_isotropy['sat_z'] = []
    x0 = systems[system]['MW_px'][0]
    y0 = systems[system]['MW_py'][0]
    z0 = systems[system]['MW_pz'][0]

    
    
    
 





    #redistribute satellites, preserving their separation vector, but new angles

    if level == 1:
        level_one_sats = np.where(systems[system]['sat_levels'] == 1)
        nsats = len(level_one_sats[0]) 
        for k in range(nsats):
            x,y,z = systems[system]['sat_pxs'][level_one_sats][k],systems[system]['sat_pys'][level_one_sats][k],systems[system]['sat_pzs'][level_one_sats][k]
            rx,ry,rz = x-x0,y-y0,z-z0
            r = np.sqrt(rx**2 + ry**2 + rz**2)
            
            cos_theta,sin_theta, phi = rand_angle()
            
            xp = r*np.cos(phi)*sin_theta + x0
            yp = r*np.sin(phi)*sin_theta + y0
            zp = r*cos_theta + z0

            spherical_isotropy['sat_x'].append(xp)
            spherical_isotropy['sat_y'].append(yp)
            spherical_isotropy['sat_z'].append(zp)


    elif level == 2:
        level_two_sats = np.where(systems[system]['sat_levels'] == 2)
        nsats = len(level_two_sats[0])

        for k in range(nsats):
            x,y,z = systems[system]['sat_pxs'][level_two_sats][k],systems[system]['sat_pys'][level_two_sats][k],systems[system]['sat_pzs'][level_two_sats][k]
            rx,ry,rz = x-x0,y-y0,z-z0
            r = np.sqrt(rx**2 + ry**2 + rz**2)
            
            cos_theta,sin_theta, phi = rand_angle()
            
            xp = rx*np.cos(phi)*sin_theta + x0
            yp = ry*np.sin(phi)*sin_theta + y0 
            zp = rz*cos_theta + z0

            spherical_isotropy['sat_x'].append(xp)
            spherical_isotropy['sat_y'].append(yp)
            spherical_isotropy['sat_z'].append(zp)


    elif level == 3:
        level_three_sats = np.where(systems[system]['sat_levels'] == 3)
        nsats = len(level_three_sats[0])

        for k in range(nsats):
            x,y,z = systems[system]['sat_pxs'][level_three_sats][k],systems[system]['sat_pys'][level_three_sats][k],systems[system]['sat_pzs'][level_three_sats][k]
            rx,ry,rz = x-x0,y-y0,z-z0
            r = np.sqrt(rx**2 + ry**2 + rz**2)
            
            cos_theta,sin_theta, phi = rand_angle()
            
            xp = rx*np.cos(phi)*sin_theta + x0
            yp = ry*np.sin(phi)*sin_theta + y0
            zp = rz*cos_theta + z0

            spherical_isotropy['sat_x'].append(xp)
            spherical_isotropy['sat_y'].append(yp)
            spherical_isotropy['sat_z'].append(zp)

    else:

        for k in range(len(systems[system]['sat_pxs'])):
            x,y,z = systems[system]['sat_pxs'][k],systems[system]['sat_pys'][k],systems[system]['sat_pzs'][k]
            rx,ry,rz = x-x0,y-y0,z-z0
            r = np.sqrt(rx**2 + ry**2 + rz**2)
            
            cos_theta,sin_theta, phi = rand_angle()
            
            xp = rx*np.cos(phi)*sin_theta + x0 
            yp = ry*np.sin(phi)*sin_theta + y0 
            zp = rz*cos_theta + z0

            spherical_isotropy['sat_x'].append(xp)
            spherical_isotropy['sat_y'].append(yp)
            spherical_isotropy['sat_z'].append(zp)


    return spherical_isotropy['sat_x'],spherical_isotropy['sat_y'],spherical_isotropy['sat_z']




In [37]:
sx,sy,sz = rand_spherical_dist(syst,level=1)


In [38]:
## check if things are being redistributed by checking the first redistribution


fig = plt.figure(figsize=[8,6])
ax = plt.axes(projection='3d')
scaleby = 100




sat_iso = ax.scatter3D(sx ,sy ,sz,
                       s=systems[syst]['sat_rvirs']*scaleby**2,c='indianred',edgecolors='red',alpha=0.4,label='Redistributed Satellites')
imsats = ax.scatter3D(systems[syst]['sat_pxs'],systems[syst]['sat_pys'],systems[syst]['sat_pzs'],
                      s=systems[syst]['sat_rvirs']*scaleby**2,c=np.log10(systems[syst]['sat_mvirs']),edgecolors='indianred',alpha=0.4,label='Original Satellites')

imcentral = ax.scatter3D(systems[syst]['MW_px'],systems[syst]['MW_py'],systems[syst]['MW_pz'],
                      s=systems[syst]['MW_rvir']*scaleby**2,c='slateblue',edgecolors='darkblue',alpha=0.4,label='Central')

central = Circle((systems[syst]['MW_px'], systems[syst]['MW_py']), radius=systems[syst]['MW_rvir'],color='slateblue',alpha=0.4,label='Rvir of MW')
ax.add_patch(central)
art3d.pathpatch_2d_to_3d(central, systems[syst]['MW_pz'], zdir="z")


#ax.set_title(r'MW type Satellite System, $N_{sats}$ = 7',y=1.15)

ax.autoscale('False')
ax.set_xlabel('X [Mpc]')
ax.set_ylabel('Y [Mpc]')
ax.set_zlabel('Z [Mpc]')
plt.legend(loc="upper left", markerscale=.4)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
## check if things are being redistributed by checking the first redistribution


fig = plt.figure(figsize=[8,6])
ax = plt.axes(projection='3d')

M_to_k = 1000
MW_x,MW_y,MW_z = systems[syst]['MW_px'],systems[syst]['MW_py'],systems[syst]['MW_pz']
nsats = len(systems[syst]['sat_pxs'])

scaleby = 4


for i in range(nsats):
    sat = Circle(((systems[syst]['sat_pxs'][i]-MW_x)*M_to_k, (systems[syst]['sat_pys'][i] - MW_y)*M_to_k), radius=systems[syst]['sat_rvirs'][i]*M_to_k,color='indianred',alpha=0.4)
    ax.add_patch(sat)
    art3d.pathpatch_2d_to_3d(sat, (systems[syst]['sat_pzs'][i]-MW_z)*M_to_k, zdir="z")

imsats = ax.scatter3D((systems[syst]['sat_pxs'] - MW_x)*M_to_k,(systems[syst]['sat_pys']- MW_y)*M_to_k,(systems[syst]['sat_pzs'] - MW_z)*M_to_k,
                      s=systems[syst]['sat_rvirs']*M_to_k*scaleby**2,c=np.log10(systems[syst]['sat_mvirs']),edgecolors='indianred',alpha=0.4,label='Satellites')
imcentral = ax.scatter3D((systems[syst]['MW_px']- MW_x)*M_to_k,(systems[syst]['MW_py']- MW_y)*M_to_k ,(systems[syst]['MW_pz'] - MW_z)*M_to_k,
                      s=systems[syst]['MW_rvir']*M_to_k*scaleby**2,c='slateblue',edgecolors='darkblue',alpha=0.4,label='Central')

central = Circle(((systems[syst]['MW_px'] - MW_x)*M_to_k, (systems[syst]['MW_py'] - MW_y)*M_to_k), radius=systems[syst]['MW_rvir']*M_to_k,color='slateblue',alpha=0.4,label='Rvir of MW')
ax.add_patch(central)
art3d.pathpatch_2d_to_3d(central, (systems[syst]['MW_pz']-MW_z)*M_to_k, zdir="z")


sat_iso = ax.scatter3D((sx - MW_x)*M_to_k,(sy - MW_y)*M_to_k,(sz  - MW_z)*M_to_k,
                      s=systems[syst]['sat_rvirs']*M_to_k*scaleby**2,c='indianred',edgecolors='red',alpha=0.4,label='Redistributed Satellites')




#ax.set_title(r'MW type Satellite System, $N_{sats}$ = 7',y=1.15)
plt.colorbar(imsats,label=r'Mass of Satellites [$log_{10} M_\odot$]')
ax.autoscale('False')
ax.set_xlabel('X [kpc]')
ax.set_ylabel('Y [kpc]')
ax.set_zlabel('Z [kpc]')
plt.legend(loc="upper left", markerscale=.4)

extent = 400 #kpc
ax.set_xlim(-extent,extent)
ax.set_ylim(-extent,extent)
ax.set_zlim(-extent,extent)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
## check that it's uniformly dist by running n times

n = 2000
rand_systems = {}
rand_systems['systems'] = []



for i in range(n):
    rand_system = {}
    #rand_system['sat_px'] = []
    #rand_system['sat_py'] = []
    #rand_system['sat_pz'] = []
    rand_system['MW_px'] = systems[syst]['MW_px'][0]
    rand_system['MW_py'] = systems[syst]['MW_py'][0]
    rand_system['MW_pz'] = systems[syst]['MW_pz'][0]
    
    sx,sy,sz = rand_spherical_dist(syst,level=1)

    
    rand_system['sat_px'] = sx
    rand_system['sat_py'] = sy
    rand_system['sat_pz'] = sz
    rand_systems['systems'].append(rand_system)
    
    


In [41]:
rand_systems['systems'][1]

{'MW_px': -2.7792458626216034,
 'MW_py': -0.2669354386349215,
 'MW_pz': -9.50933860865193,
 'sat_px': [-2.144663165882095,
  -2.76535997418471,
  -2.780992808003095,
  -2.837356324137791,
  -3.063104343464935,
  -2.972732615612048,
  -2.743139370688407,
  -2.717445826767088,
  -2.810932945919906,
  -2.806711928082439,
  -2.669752337540977,
  -2.728574341140186,
  -2.815461285364395,
  -2.7565430996690745],
 'sat_py': [-0.5558798638099525,
  -0.18981617858823766,
  -0.30656096599767124,
  -0.34838444029196486,
  -0.11759683811283303,
  0.039012676576319405,
  -0.45286081021780267,
  -0.6688080901664497,
  -0.24619494818216237,
  -0.2559918120869566,
  -0.25978143210038857,
  -0.27048660667859975,
  -0.24368155597056812,
  -0.3280314363084684],
 'sat_pz': [-9.640362896969743,
  -9.395121767270476,
  -9.583354113123622,
  -9.45369919017685,
  -9.844609540904578,
  -9.771851385748793,
  -9.532265590268135,
  -9.537975340974002,
  -9.4596851358457,
  -9.40133248108189,
  -9.387353605059284,

In [42]:
norm = matplotlib.colors.Normalize(vmin=0, vmax=n)
colors = [(cm.plasma(norm(i))) for i in range(n)]

fig = plt.figure(figsize=[8,8])
ax = plt.axes(projection='3d')

for system in range(n):
    sphere_points = ax.scatter3D((rand_systems['systems'][system]['sat_px']-MW_x)*M_to_k,(rand_systems['systems'][system]['sat_py']-MW_y)*M_to_k,(rand_systems['systems'][system]['sat_pz']-MW_z)*M_to_k,c= np.atleast_2d(colors[system]),alpha=0.2)

#"""    
extent = 400
ax.set_xlim(-extent,extent)
ax.set_ylim(-extent,extent)
ax.set_zlim(-extent,extent)
#"""

plt.show()
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Collect Cos(theta) For All Planes

In [43]:
t0 = time.time()
cos_thetas = []
mean_rms = []

for rand_syst in range(n):
    
    best_u1,best_u2,best_u3,nx,ny,nz,rms,rand_rms,delta_s,best_rand_cos_theta = best_plane(system=rand_systems['systems'][rand_syst],n=2000,rand=True)
    cos_thetas.append(best_rand_cos_theta)
    mean_rms.append(rand_rms)
    
t1 = time.time()
print('Total time:', t1-t0, 'seconds')

Total time: 397.9521770477295 seconds


In [65]:
print(len(mean_rms))

2000


In [44]:
# Creating histogram
#angles = np.asarray(angles)

fig, ax = plt.subplots(1, 1,
                        figsize =(8,5), 
                        tight_layout = True)

histbins = 50
  
ax.hist(cos_thetas, density=False,bins =histbins,ec='purple',fc='thistle')
ax.axvline(x=best_cos_theta,c='black',label=r'cos($\theta$) of actual distribution')
ax.set_title(f'Angular Distribution of {n} Isotropically Distributed Planes')
ax.set_xlabel(r'(Cos $\theta$)')
ax.legend()
  
# Show plot
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
# Creating histogram
#angles = np.asarray(angles)

fig, ax = plt.subplots(1, 1,
                        figsize =(8,5), 
                        tight_layout = True)

histbins = 70
  
ncounts,dense_bins,patches = ax.hist(mean_rms, density=True,bins =histbins,ec='purple',fc='thistle')
ax.axvline(x=best_rms,c='black',label='mean rms of actual distribution')

mu, sigma = ss.norm.fit(mean_rms)
best_fit_line = ss.norm.pdf(dense_bins, mu, sigma)
ax.plot(dense_bins,best_fit_line,c='purple',label='PDF')

ax.set_title(f'Distribution of {n} Isotropically Distributed Planes')
ax.set_xlabel(r'Mean RMS')
ax.legend()
  
# Show plot
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …